# Networks Datacamp 2

### Exploratory data analysis
Here, a graph G has been loaded into your workspace. Your task is to perform some exploratory analysis in the IPython Shell. What is the type of this graph? How many nodes and edges are present?

The .nodes() and .edges() methods will be useful to you here, as will the type() and len() functions. Using these, you can start describing the graph's basic statistics.

### Plotting using nxviz
Now, you're going to practice creating a CircosPlot using nxviz! As a bonus preview of what's coming up in the next video, there's a little segment on the bipartite keyword in this exercise!

Here, the degree centrality score of each node has been added to their metadata dictionary for you using the following code:

```
# Add the degree centrality score of each node to their metadata dictionary
dcs = nx.degree_centrality(G)
for n in G.nodes():
    G.node[n]['centrality'] = dcs[n]
```

In [ ]:
# Create the CircosPlot object: c
c = CircosPlot(G, node_color='bipartite', node_grouping='bipartite', node_order='centrality')

# Draw c to the screen
c.draw()

# Display the plot
plt.show()

## Bipartite graphs
- Graphs partitioned into 2 sets
- Nodes can only be connected to nodes in other partition
- Opposite of unipartite graphs
- Networkx does not have an explicit bipartite graph, but you can filter nodes to get the same effect

### Bipartite keywork
In the video, Eric introduced you to the 'bipartite' keyword. This keyword is part of a node's metadata dictionary, and can be assigned both when you add a node and after the node is added. Remember, though, that by definition, in a bipartite graph, a node cannot be connected to another node in the same partition.

Here, you're going to write a function that returns the nodes from a given partition in a bipartite graph. In this case, the relevant partitions of the Github bipartite graph you'll be working with are 'projects' and 'users'.